# Section 2-3 - Convolutional Neural Network

The approach we've taken so far treats each image a single 'flat' vector of length 784. In this section, we'll introduce layers that take advantage the 2D structure of each 28x28 MNIST image, helping simplify computation and improve model performance.

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

df = pd.read_csv('data/mnist.csv')

In [2]:
df_train = df.iloc[:33600, :]

X_train = df_train.iloc[:, 1:].values / 255.
y_train = df_train['label'].values
y_train_onehot = pd.get_dummies(df_train['label']).values

In [3]:
df_test = df.iloc[33600:, :]

X_test = df_test.iloc[:, 1:].values / 255.
y_test = df_test['label'].values

## Benchmark

In [4]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, df_train['label'].values)

y_prediction = model.predict(X_test)
print("\naccuracy", np.sum(y_prediction == df_test['label'].values) / float(len(y_test)))

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 100


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.3s remaining:    0.0s


building tree 2 of 100


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.7s remaining:    0.0s


building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100
building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:   29.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s



accuracy 0.965


[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.5s finished


## CNN

The first layer we'll introduce is called the convolutional layer. Instead of having a weight matrix of shape (output length x input length), we'll instead consider a 3x3 weight matrix called a filter or kernel. We take the vector product of the filter with each (overlapping) 3x3 grid in the 28x28 image.

Since there are 26x26 such grids, a single filter results in a 26x26 output. 32 filters gives us an output 'volume' of shape 26x26x32. By making the 26x26x32 volume go through another convolutional layer, we end up with an output volume of shape 24x24x32.

The second new layer we'll use is called the pooling layer. Here we divide up the input into non-overlapping grids of size 2x2, and take the maximum value from each grid. For an 24x24x32 input, this results in 12x12x32 output. This volume is then 'flattened' to a vector of length 4,608, which we manipulate the same way as any vector.

The use of filters constraints the architecture of the network as each filter only focuses on a specific aspect of the data. This allows our model to scale better and be more translation-invariant. The pooling layer also reduces the number of parameters, helping reduce computation and limit overfitting.

An excellent detailed discussion, with architectural variations and helpful illustrations, can be found on Stanford's CS231n course notes:

http://cs231n.github.io/convolutional-networks/

In [5]:
# https://github.com/fchollet/keras/blob/master/examples/mnist_cnn.py

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, MaxPooling2D, Flatten
from keras.losses import categorical_crossentropy
from keras.optimizers import RMSprop

start = time()

img_rows, img_cols = 28, 28
nb_filters = 32
pool_size = (2, 2)
kernel_size = (3, 3)

X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

model = Sequential()
model.add(Conv2D(nb_filters, kernel_size=kernel_size,
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(nb_filters, kernel_size, activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

model.compile(loss=categorical_crossentropy,
              optimizer=RMSprop(),
              metrics=['accuracy'])

model.fit(X_train, y_train_onehot, epochs=12)

model.summary()

print('\ntime taken %s seconds' % str(time() - start))

Epoch 1/12
1050/1050 [==============================] - 61s 58ms/step - loss: 0.2487 - accuracy: 0.9239
Epoch 2/12
1050/1050 [==============================] - 63s 60ms/step - loss: 0.1015 - accuracy: 0.9699
Epoch 3/12
1050/1050 [==============================] - 71s 67ms/step - loss: 0.0851 - accuracy: 0.9761
Epoch 4/12
1050/1050 [==============================] - 68s 65ms/step - loss: 0.0734 - accuracy: 0.9779
Epoch 5/12
1050/1050 [==============================] - 85s 81ms/step - loss: 0.0705 - accuracy: 0.9790
Epoch 6/12
1050/1050 [==============================] - 71s 67ms/step - loss: 0.0663 - accuracy: 0.9813
Epoch 7/12
1050/1050 [==============================] - 72s 68ms/step - loss: 0.0677 - accuracy: 0.9808
Epoch 8/12
1050/1050 [==============================] - 75s 71ms/step - loss: 0.0669 - accuracy: 0.9808
Epoch 9/12
1050/1050 [==============================] - 70s 66ms/step - loss: 0.0670 - accuracy: 0.9812
Epoch 10/12
1050/1050 [==============================] - 67s 63m

In [6]:
y_prediction = np.argmax(model.predict(X_test), axis=-1)
print("\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test)))


accuracy 0.9833333333333333


This is our best result yet! It is worth experimenting with different network architectures to see how they affect loss and accuracy. In the next section, we'll continue to take advantage of inherent data structures but apply it to text data.